# 과제: 네이버 영화 정보 및 평점 크롤링

- 대상: 예매순 상위 5개의 현재 상영 중인 영화
- 수집할 항목: 영화 제목, 주연배우 3인, 네티즌 평점, 관람객 평점, 기자/평론가 평점, 관람객 별점 리뷰 20건 공감순으로(평점, 작성자닉네임, 리뷰본문)

In [1]:
#셀레니움을 사용하기 위한 정보를 다운로드 받습니다. 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.wait import WebDriverWait as Wait
import pandas as pd
import numpy as np
import re
import time

### 1. 예매순 상위 5개의 현재 상영 중인 영화 가져오기
영화 제목, 주연배우 3인

In [2]:
driver = webdriver.Chrome("./chromedriver.exe") #웹드라이브 실행
driver.get("https://movie.naver.com/movie/running/current.nhn") #주소 get
titles = Wait(driver, 5).until(
        ec.presence_of_all_elements_located((By.CSS_SELECTOR, "dt.tit a")))
title_li, movie_url_li = list(zip(*[(title.text, title.get_attribute("href")) 
                                    for index, title in enumerate(titles) if index < 5]))

In [3]:
# 현재 상영 영화들 중 랭킹 1~5위 영화 관련 제목, url, 배우 정보 스크래핑
def movie_title_url_actor(driver):
    driver.get("https://movie.naver.com/movie/running/current.nhn")
    
    # 현재 상영 영화들 중 랭킹 1~5위 영화의 제목과 해당 영화 정보로 넘어가는 url 스크래핑
    titles = Wait(driver, 5).until(
        ec.presence_of_all_elements_located((By.CSS_SELECTOR, "dt.tit a")))
    title_li, movie_url_li = list(zip(*[(title.text, title.get_attribute("href"))
                                        for index, title in enumerate(titles) if index < 5])) #제목과 url 저장

    # 영화 주연 각 영화별로 3명까지 스크래핑
    total_actors_li = []

    for i in range(1, 6):
        actors = Wait(driver, 5).until(
            ec.presence_of_all_elements_located((By.CSS_SELECTOR, "li:nth-of-type(" + str(i) + ") dl.info_txt1 dd:nth-of-type(3) span.link_txt a"))) 
        actors_li = [actor.text for index, actor in enumerate(actors) if index < 3]
        total_actors_li.append(actors_li)
        
    return movie_url_li, title_li, total_actors_li

### 2. 해당 영화의 평점 가져오기
네티즌 평점, 관람객 평점, 기자/평론가 평점

In [4]:
# 네티즌 평점, 관람객 평점, 기자/평론가 평점 평균 스크래핑
def get_grade(driver):
    viewer_score = Wait(driver, 5).until(
        ec.presence_of_element_located((By.XPATH, '//*[@id="actualPointPersentBasic"]')))
    viewer_score = viewer_score.text
    viewer_score = "".join(re.findall("(\d+\.\d+)점", viewer_score))

    critic_score = Wait(driver, 5).until(
        ec.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[2]/div[1]/div[1]/div[2]/div/a/div')))
    critic_score = critic_score.text

    netizen_score = Wait(driver, 5).until(
        ec.presence_of_element_located((By.XPATH, '//*[@id="pointNetizenPersentBasic"]')))
    netizen_score = netizen_score.text

    return (viewer_score, critic_score, netizen_score)

In [5]:
# 리뷰 관련 정보 스크래핑
def get_user_review_info(driver): 
    # 리뷰 더 보기 클릭
    more_comments_bt = Wait(driver, 5).until(
    ec.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/div[4]/div[5]/div[2]/div[1]/a')))

    more_comments_bt.click()

    # 리뷰 블록 url 가져오기
    comments_block_url = Wait(driver, 5).until(
        ec.presence_of_element_located((By.XPATH, '//*[@id="pointAfterListIframe"]')))
    comments_block_url = comments_block_url.get_attribute("src")

    driver.get(comments_block_url)
    
    # 리뷰 스크래핑
    for count in range(2):
        # 스포일러 댓글 전부 오픈하기
        try:
            spoiler_open_bts = Wait(driver, 1).until(
                ec.presence_of_all_elements_located((By.CSS_SELECTOR, "span.text_spo a.btn_more")))     
        except:
            pass
        else:
            for spoiler_open_bt in spoiler_open_bts:
                spoiler_open_bt.click()
        
        # 가려진 댓글 전부 오픈하기
        try:
            unfolding_bts = Wait(driver, 1).until(
                ec.presence_of_all_elements_located((By.CSS_SELECTOR, "div.score_result div.score_reple p span span a")))
        except:
            pass
        else:
            for unfolding_bt in unfolding_bts:
                unfolding_bt.click()
                
        scores = Wait(driver, 5).until(
            ec.presence_of_all_elements_located((By.CSS_SELECTOR, "div.score_result div.star_score em")))
        
        nicknames = Wait(driver, 5).until(
            ec.presence_of_all_elements_located((By.CSS_SELECTOR, "div.score_reple dl dt em a span")))
   
        reviews = []
        for review_order in range(1, 11):
            try:
                review = Wait(driver, 1).until(
                    ec.presence_of_all_elements_located((By.CSS_SELECTOR, "div.score_result ul li:nth-of-type(" + str(review_order) + ") div.score_reple p span")))
                # "관람객" 같은 불필요한 정보가 나오지 않는 가장 마지막 원소를 추가함
                reviews.append(review[-1])
            except:
                pass

        next_bt = Wait(driver, 5).until(
            ec.presence_of_element_located((By.CSS_SELECTOR, "div.paging div a:last-of-type")))

        if count == 0:
            score_li = [score.text for score in scores]
            nickname_li = [nickname.text for nickname in nicknames]
            review_li = [review.text for review in reviews]

            next_bt.click()
            driver.implicitly_wait(0.5)
        else:
            score_li.extend([score.text for score in scores])
            nickname_li.extend([nickname.text for nickname in nicknames])
            review_li.extend([review.text for review in reviews])
    return (score_li, nickname_li, review_li)

### 3. 관람객 평점 공감순 20건 가져오기
평점, 평점 작성자 닉네임, 리뷰 본문

In [6]:
# 영화에 대한 반응(평점, 리뷰) 관련 정보 스크래핑
def get_reviews(driver, movie_url_li, movie_title_li, movie_actors_li):
    total_title_li = []
    total_actors_li = []
    total_viewer_score = []
    total_critic_score = []
    total_netizen_score = []
    total_score_li = []
    total_nickname_li = []
    total_review_li = []

    for movie_url, movie_actors, movie_title in zip(movie_url_li, movie_actors_li, movie_title_li):
        driver.get(movie_url)
        viewer_score, critic_score, netizen_score = get_grade(driver)
        score_li, nickname_li, review_li = get_user_review_info(driver)


        total_title_li.extend([movie_title] * len(score_li))
        total_actors_li.extend([movie_actors] * len(score_li))
        total_viewer_score.extend([viewer_score] * len(score_li))
        total_critic_score.extend([critic_score] * len(score_li))
        total_netizen_score.extend([netizen_score] * len(score_li))
        total_score_li.extend(score_li)
        total_nickname_li.extend(nickname_li)
        total_review_li.extend(review_li)
        
    movie_info_dict = {"제목": total_title_li, "주연배우": total_actors_li, "관람객평점": total_viewer_score,
                       "기자/평론가평점": total_critic_score, "네티즌평점": total_netizen_score,
                       "댓글평점": total_score_li, "닉네임": total_nickname_li, "댓글내용": total_review_li}
        
    return movie_info_dict

In [7]:
driver = webdriver.Chrome("./chromedriver.exe")
driver.implicitly_wait(0.5)

movie_url_li, movie_title_li, movie_actors_li =  movie_title_url_actor(driver)
movie_info_dict = get_reviews(driver, movie_url_li, movie_title_li, movie_actors_li)
driver.close()

In [8]:
movie_info_df = pd.DataFrame(movie_info_dict)

In [9]:
movie_info_df.head()

,제목,주연배우,관람객평점,기자/평론가평점,네티즌평점,댓글평점,닉네임,댓글내용
0,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,10,bohemian(mabu****),"난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 ..."
1,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,1,어쩌라고(dpfk****),아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽...
2,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,10,최정규(cjg4****),전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
3,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,10,달다(fxko****),8명의 배우가 모두 주인공 같은 느낌.
4,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,9,써니(tlag****),개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ


In [10]:
movie_info_df["아이디"] = movie_info_df["닉네임"]

movie_info_df["닉네임"] = list(map(lambda x: "".join(re.findall("(.+)\(", x)), movie_info_df["닉네임"]))
movie_info_df["아이디"] = list(map(lambda x: "".join(re.findall("\w+\*+", x)), movie_info_df["아이디"]))

In [11]:
movie_info_df2 = movie_info_df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 8, 7]]
movie_info_df2.to_csv("네이버 영화 리뷰.csv", index=False, encoding="utf-8-sig")

In [12]:
movie_info_df2.head()

,제목,주연배우,관람객평점,기자/평론가평점,네티즌평점,댓글평점,닉네임,아이디,댓글내용
0,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,10,bohemian,mabu****,"난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 ..."
1,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,1,어쩌라고,dpfk****,아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽...
2,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,10,최정규,cjg4****,전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
3,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,10,달다,fxko****,8명의 배우가 모두 주인공 같은 느낌.
4,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,9,써니,tlag****,개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ


In [13]:
driver.quit()

In [14]:
movie_info_df2

,제목,주연배우,관람객평점,기자/평론가평점,네티즌평점,댓글평점,닉네임,아이디,댓글내용
0,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,10,bohemian,mabu****,"난 전도연의 화류계 캐릭터가 좋다. 무뢰한, 너는 내 운명, 카운트다운...그리고 ..."
1,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,1,어쩌라고,dpfk****,아니 개봉당일날 9시 땡하고 부터 평점 쏟아지는게 말이 돼냐? 요즘 조조는 꼭두새벽...
2,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,10,최정규,cjg4****,전도연 연기 진짜 오진다...와 이 영화에서 완전 섹시하게 나온다 역시 명불허전임...
3,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,10,달다,fxko****,8명의 배우가 모두 주인공 같은 느낌.
4,지푸라기라도 잡고 싶은 짐승들,"[전도연, 정우성, 배성우]",8.39,6.71,6.85,9,써니,tlag****,개존잼 역시 전도연이죠? 카리스마 미쳐벌여ㅠㅁㅠ
...,...,...,...,...,...,...,...,...,...
95,클로젯,"[하정우, 김남길, 허율]",8.39,5.50,6.86,1,지애,maln****,최근에 본 것 중에 제일 최악..
96,클로젯,"[하정우, 김남길, 허율]",8.39,5.50,6.86,10,삡,jiny****,김남길 눈빛이 다했다..재밌어요
97,클로젯,"[하정우, 김남길, 허율]",8.39,5.50,6.86,10,maimai,enma****,진짜 심장 쫄깃하고 무서움 ㄷㄷㄷ 이건 극장에서 봐야함
98,클로젯,"[하정우, 김남길, 허율]",8.39,5.50,6.86,10,난꽃이다,play****,무서운거 진짜 못보는대 혼자 봤거든요 걱정하시는분들 그걸 감안하고라도 보시는거 추천...
